In [1]:
import datetime
import QuantLib as ql
from module.Swap_Curve import GET_QUOTE, SWAP_CURVE # 이전에 만든 함수들 가져오기

In [2]:
class IRS():
    # __init__는 class 초기화(내부화) 기능(class 내부에서 변수를 처리할 수 있도록 만듬?)
    def __init__(self, date, effective_date, maturity_date, irs_rate, notional, position, spread=0.0): # spread는 기본값 0으로 지정
        
        # Initial Setup 1 : Date & Curve
        self.date = date # 평가 일자
        self.curve = self.CURVE(self.date) # class 내부에서 self.curve는 todays_date에 커브의 값을 지정함
        
        # Initial Setup 2 : Instrument Info
        self.effective_date = ql.Date(effective_date.day, effective_date.month, effective_date.year) # 퀀트립 형태의 날짜로 바꿈
        self.maturity_date = ql.Date(maturity_date.day, maturity_date.month, maturity_date.year) # 퀀트립 형태의 날짜로 바꿈
        self.calendar = ql.UnitedStates() # 달력(미국 기준으로)
        self.convention = ql.ModifiedFollowing # 휴일 처리(ModifiedFollowing 방식)
        self.day_counter = ql.Actual360() # 날짜 카운팅(Actual360 ==> 1년을 360일로(미국 기준으로))
        self.fixed_tenor = ql.Period(1, ql.Years) # 고정 금리(통상 1년마다 지불하므로 기간을 1년으로 설정)
        self.float_tenor = ql.Period(3, ql.Months) # 변동 금리(통상 3개월마다 지불하므로 기간을 3개월으로 설정)
        self.irs_rate = irs_rate # 고정 금리
        self.notional = notional # 명목 금액
        self.spread = spread
        if position == 'Long':
            self.position = ql.VanillaSwap.Payer # 선도 금리 계약의 포지션을 Long으로 지정
        else:
            self.position = ql.VanillaSwap.Receiver # 선도 금리 계약의 포지션을 Short로 지정
        
        # Pricing Results(나중에 구현할 함수를 미리 지정해놓음)
        
        self.npv = self.PRICING(self.curve) # 이론가 계산할 수 있는 함수
        self.delta = self.DELTA() # 금리 커브 변화에 따른 포지션의 변화를 계산할 수 있는 델타 함수
        self.theta = self.THETA() # 시장 환경은 변화하지 않았지만, 시간이 변할 때 포지션 변화를 계산할 수 있는 세타 함수
        
    def CURVE(self, date):
        return SWAP_CURVE(date, GET_QUOTE(date))
    
    def PRICING(self, curve):
        # Yield Term-structure
        curve_handle = ql.YieldTermStructureHandle(curve) # 커브 핸들 변수 생성
        
        # USD 3M LIBOR
        float_index = ql.USDLibor(ql.Period(3, ql.Months), curve_handle) # 준거 금리 변수 생성(여기서는 3개월로 지정함)
        # float_index.addFixing(ql.Date(9, 10, 2020), 0.0022538) -> 10월 9일의 금리 값을 지정해줌(이러면 효력발생일을 10.19로 안해도 됨)
        
        # 고정 금리 스케쥴
        fixedSchedule = ql.Schedule(self.effective_date, # 정산일
                                    self.maturity_date, # 만기일자
                                    self.fixed_tenor, # 고정 금리
                                    self.calendar, # 달력
                                    self.convention, # 만기일이 아닌 중간중간 정산 날의 휴일 처리
                                    self.convention, # 실제 만기일의 휴일 처리
                                    ql.DateGeneration.Backward, # 정산일부터? 만기일부터?
                                    False) # Backward 만기일부터 일정 주기만큼 앞으로 오면서 이자 지급을 결정
        
        # 변동 금리 스케쥴
        floatingSchedule = ql.Schedule(self.effective_date, # 정산일
                                       self.maturity_date, # 만기일자
                                       self.float_tenor, # 고정 금리
                                       self.calendar, # 달력
                                       self.convention, # 만기일이 아닌 중간중간 정산 날의 휴일 처리
                                       self.convention, # 실제 만기일의 휴일 처리
                                       ql.DateGeneration.Backward,
                                       False) # Backward 만기일부터 일정 주기만큼 앞으로 오면서 이자 지급을 결정
        
        # Interes Rate Swap
        irs = ql.VanillaSwap(self.position, # 포지션(매매 방향)
                             self.notional, # 명목 금액
                             fixedSchedule, # 고정 금리 스케쥴
                             self.irs_rate, # 고정 금리
                             self.day_counter, # 날짜 카운팅(고정 금리)
                             floatingSchedule, # 변동 금리 스케쥴
                             float_index, # 준거 금리(변동 금리의 준거 금리)
                             self.spread, # 0으로 지정한 값
                             self.day_counter) # 날짜 카운팅(변동 금리)

        # Pricing Engine
        swapEngine = ql.DiscountingSwapEngine(curve_handle)
        irs.setPricingEngine(swapEngine) # irs를 swapEngine으로 평가하는 것
        npv = irs.NPV() # NPV() ==> fra의 이론적인 가치를 구하고 싶을 때
        return npv
    
    def DELTA(self): # 금리 커브가 1bp 움직였을 때, IRS의 가격이 얼마나 변하는지
        curve_handle = ql.YieldTermStructureHandle(self.curve) # 커브 핸들 변수 생성(PRICING과 같음)
        
        # 1bp
        basis_point = 0.0001 # 0.01%를 의미
        
        # FRA price when 1bp up
        up_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(basis_point)))
        # ZeroSpreadedTermStructure ==> 기존의 기간 구조를 받은 후 몇 bp 위 or 아래로 움직여줄 것인지를 구현하는 함수
        up_irs = self.PRICING(up_curve) # +1bp 움직였을 때, Pricing
        
        # FRA price when 1bp down
        down_curve = ql.ZeroSpreadedTermStructure(curve_handle, ql.QuoteHandle(ql.SimpleQuote(-basis_point)))
        down_irs = self.PRICING(down_curve) # -1bp 움직였을 때, Pricing
        
        # Delta 계산
        dv01 = (up_irs - down_irs) / 2
        
        return dv01
    
    def THETA(self): # 오직 시간이 한 단위 변했을 때(시장 환경엔 변화 X), IRS의 가격이 얼마나 변하는지
        price_t0 = self.PRICING(self.CURVE(self.date)) # 기존 pricing
        price_t1 = self.PRICING(self.CURVE(self.date + datetime.timedelta(days=1))) # 하루 지난 후 pricing
        
        return price_t1 - price_t0

In [3]:
if __name__ == "__main__":

    # Today's Date
    todays_date = datetime.date(2020, 10, 9)
    
    # IRS Instrument Setup
    effective_date = datetime.date(2020, 10, 19) # 효력 발생 일
    # 통상 거래 일 2일 뒤에 효력이 발생하지만 9일의 금리 값이 없어서 19일 후로 설정(거래일은 17일이 되는 것, 17일 기준 금리 값이 들어감)
    maturity_date = datetime.date(2022, 10, 19) # 만기일
    position = 'Long'
    irs_rate = 0.00218 # 고정 금리(0.2%)
    notional = 10000000 # 명목 금액(천 만불)
    
    # Build IRS object
    irs = IRS(todays_date,
              effective_date,
              maturity_date,
              irs_rate,
              notional,
              position)
    
    print("Price = {}".format(round(irs.npv, 4)))
    print("Delta = {}".format(round(irs.delta, 4)))
    print("Theta = {}".format(round(irs.theta, 4)))

Price = 95.9499
Delta = 2025.3957
Theta = -2.9461
